# RABIES installation and usage

Here we specify the code to run RABIES on a test dataset from the StandardRat dataset (subject groups 300800-3008009 and 301400-3001409).

For running rabies preproccessing, default settings were used for all except to following arguments:
--TR 1 --commonspace_reg masking=false,brain_extraction=false,template_registration=SyN,fast_commonspace=true --commonspace_resampling 0.3x0.3x0.3

If based on QC-report this was unstatisfactory, the following arguments were tried iteretiveley: 
1. --anat_autobox --bold_autobox 
2. Option 1 + --anat_inho_cor method=N4_reg,otsu_thresh=2,multiotsu=false --bold_inho_cor method=N4_reg,otsu_thresh=2,multiotsu=false 
3. Option 2 + --bold2anat_coreg masking=false,brain_extraction=false,registration=Rigid

For using RABIES, it is recommended to build an apptainer or docker container. See https://apptainer.org/docs/user/latest/quick_start.html for the instruction to install apptainer or https://docs.docker.com/engine/install/ to install docker. For the latest version of RABIES, see https://hub.docker.com/r/gabdesgreg/rabies/tags

In [ ]:
#!/usr/bin/env bash

#install RABIES using apptainer
apptainer pull rabies.sif docker://gabdesgreg/rabies:0.4.8

#Or docker
docker pull gabdesgreg/rabies:0.4.8

The following code can be used to run rabies using the default settings with apptainer. This code will immediately run all three steps, namely preprocessing, confound correction and analysis. For the analysis we have added some of the essentials, such as creating a whole brain functional connectivity matrix and dual-regression independent component analysis

The -B flag is used to mount folders. For input folders, such as input_bids, :ro is added to only give RABIES reading permission. This prevents data being changed or removed. For output folders :ro is not added, as RABIES needs writing permission for these.

In [ ]:
#move to working directory
cd /PATH/TO/WORKING/DIRECTORY

#specify bids, e.g. /home/youruser/documents/BIDS
bids=/PATH/TO/BIDS/FOLDER

#specify RABIES container, e.g. /home/youruser/documents/RABIES.sif. This will depend on where rabies is pulled during the asset preparation
rabies=PATH/TO/RABIES/CONTAINER

#specify rat templates and prior ICA maps (RABIES defaults are mouse templates)
anat_template=/templates/SIGMA_InVivo_Anatomical_Brain_template.nii.gz
brain_mask=/templates/SIGMA_InVivo_Anatomical_Brain_mask.nii.gz
WM_mask=/templates/SIGMA_InVivo_Anatomical_wm_bin.nii.gz
CSF_mask=templates/SIGMA_InVivo_Anatomical_csf_bin.nii.gz
labels=/templates/SIGMA_InVivoAnatomical_Brain_Atlas.nii.gz
vascular_mask=/templates/SIGMA_InVivo_Anatomical_csf_bin.nii.gz
ICA_prior=/templates/canica_resting_state.nii.gz

#these functions will create empty directories for the preprocessed, confound, and analysis files. 
mkdir -p func_preprocess
mkdir -p func_confound
mkdir -p func_analysis

#apptainer call to run the preprocess step for RABIES
apptainer run -B $bids:/input_bids:ro -B templates:/templates:ro -B $func_preprocess:/preprocess_outputs/ $rabies -p MultiProc preprocess /input_bids/ /preprocess_outputs/ --anat_template $anat_template --brain_mask $brain_mask --WM_mask $WM_mask --CSF_mask $CSF_mask --labels $labels --vascular_mask $vascular_mask --TR 1 --commonspace_reg masking=false,brain_extraction=false,template_registration=SyN,fast_commonspace=false --commonspace_resampling 0.3x0.3x0.3
   
#apptainer call to run the confound removal step for RABIES
apptainer run -B $bids:/input_bids:ro -B templates:/templates:ro -B $func_preprocess:/preprocess_outputs/ -B $func_confound:/confound_correction_outputs/ $rabies confound_correction /preprocess_outputs/ /confound_correction_outputs/ --TR 1 --highpass 0.01 --smoothing_filter 0.5 --lowpass 0.1 --conf_list global_signal mot_6

#apptainer call to run the analysis step for RABIES
apptainer run -B $bids:/input_bids:ro -B templates:/templates:ro -B $func_preprocess:/preprocess_outputs/ -B $func_confound:/confound_correction_outputs/ -B $func_analysis:/analysis_outputs/ $rabies -p MultiProc analysis /confound_correction_outputs/ /analysis_outputs/ --data_diagnosis --prior_maps $ICA_prior --FC_matrix --DR_ICA --group_ica apply=true,dim=10,random_seed=1 --prior_bold_idx 0 1 3 5 7 8 9 10 11 15 17 18 --prior_confound_idx 2 4 6 12 13 14 16 19


And here is a similar example, but now for Docker. Here the -B is replaced by -v and some extra parameters (--it --rm) are needed

In [ ]:
#move to working directory
cd /PATH/TO/WORKING/DIRECTORY

#specify bids, e.g. /home/youruser/documents/BIDS
bids=/PATH/TO/BIDS/FOLDER

#specify RABIES container, e.g. /home/youruser/documents/RABIES.sif. This will depend on where rabies is pulled during the asset preparation
rabies=PATH/TO/RABIES/CONTAINER

#specify rat templates and prior ICA maps (RABIES defaults are mouse templates)
anat_template=templates/SIGMA_InVivo_Brain_Template.nii
brain_mask=templates/SIGMA_InVivo_Brain_Mask.nii
WM_mask=templates/SIGMA_InVivo_WM_bin.nii.gz
CSF_mask=templates/SIGMA_InVivo_CSF_bin.nii.gz
labels=templates/SIGMA_Anatomical_Brain_Atlas.nii
vascular_mask=templates/SIGMA_InVivo_CSF_bin.nii.gz
ICA_prior=templates/canica_resting_state.nii.gz

#these functions will create empty directories for the preprocessed, confound, and analysis files. 
mkdir -p func_preprocess
mkdir -p func_confound
mkdir -p func_analysis

#docker call to run the preprocess step for RABIES
docker run -it --rm -v $bids:/input_BIDS:ro -v templates:/templates:ro -v $func_preprocess:/preprocess_outputs/ $rabies -p MultiProc preprocess /input_BIDS/ /preprocess_outputs/ --anat_template $anat_template --brain_mask $brain_mask --WM_mask $WM_mask --CSF_mask $CSF_mask --labels $labels --vascular_mask $vascular_mask --TR 1 --commonspace_reg masking=false,brain_extraction=false,template_registration=SyN,fast_commonspace=false --commonspace_resampling 0.3x0.3x0.3
    
#docker call to run the confound removal step for RABIES
docker run -it --rm -v $bids:/input_bids:ro -v templates:/templates:ro -v $func_preprocess:/preprocess_outputs/ -v $func_confound:/confound_correction_outputs/ $rabies -p MultiProc confound_correction /preprocess_outputs/ /confound_correction_outputs/ --TR 1 --highpass 0.01 --smoothing_filter 0.5 --lowpass 0.1 --conf_list global_signal mot_6

#docker call to run the analysis step for RABIES
docker run -it --rm -v $bids:/input_bids:ro -v templates:/templates:ro -v $func_preprocess:/preprocess_outputs/ -v $func_confound:/confound_correction_outputs/ -v $func_analysis:/analysis_outputs/ $rabies -p MultiProc analysis /confound_correction_outputs/ /analysis_outputs/ --data_diagnosis --prior_maps $ICA_prior --FC_matrix --DR_ICA --group_ica apply=true,dim=10,random_seed=1 --prior_bold_idx 0 1 3 5 7 8 9 10 11 15 17 18 --prior_confound_idx 2 4 6 12 13 14 16 19